In [1]:
import numpy as np
import pandas as pd

# Modification functions

In [2]:
# Hónapokból évszakok kiszámítása
def get_season(mm=""):
    if type(mm)==float:
        return None
    else:
        mm=int(mm)
    seasons={
        "1": [12,1,2], #winter
        "2": [3,4,5],  #spring
        "3": [6,7,8],  #summer
        "4": [9,10,11] #autumn
    }
    for season in seasons.keys():
        if mm in seasons[season]:
            return season
        
        
def calc_duration(fromDate, toDate):
    try: 
        return (pd.Timestamp(fromDate) - pd.Timestamp(toDate)).days
    except:
        print(fromDate, toDate)
        return None
    

def mod_df(data = pd.DataFrame()):
    
    # Keresési dátumok szétbontása év, hónap, napra
    data[["srch_ci_year", "srch_ci_mm", "srch_ci_day"]] = data["srch_ci"].str.split("-", expand=True)
    
    data['srch_ci_season']=data['srch_ci_mm'].apply(lambda x: get_season(x))
    
    data['srch_ppl_cnt'] = data.apply(lambda x: (x['srch_adults_cnt'] + x['srch_children_cnt']),axis=1)
    
    data['stay_duration_day'] = data.apply(lambda x: calc_duration(x['srch_co'], x['srch_ci']),axis=1)
    
    data['search_duration_day'] = data.apply(lambda x: calc_duration(x['srch_ci'], x['date_time']), axis=1)
    
    '''#Ezeket majd csak ott törlöm, ahol nem kellenek
    columns = ['date_time','user_id','user_location_country','srch_adults_cnt', 
               'channel','site_name','srch_children_cnt','user_location_region', 
               'srch_co','posa_continent','hotel_continent','srch_destination_type_id',
               'srch_ci','orig_destination_distance','srch_ci_year','srch_ci_day','srch_ci_mm']
    data = data.drop(columns, axis=1)'''
    
    data = data.drop(['date_time', 'srch_ci', 'srch_co'], axis=1)
    
    data = data.fillna(0)
    data = data.astype('int')
    
    return data

## Remove dirty rows

In [3]:
with open('data/train.csv', 'r') as rf:
    with open('data/train_clean.csv', 'w') as wf:
        for line in rf:
            if ',,' not in line and ',\n' not in line:
                wf.write(line)
                
train_clean = pd.read_csv('data/train_clean.csv', delimiter = ',')
print("Length of the cleaned train data: {}".format(len(train_clean)))

Length of the cleaned train data: 24117894


### Remove not booked rows

In [4]:
train_clean = mod_df(train_clean)

In [5]:
train_clean.to_csv('data/train_clean_mod.csv', sep=',', index=False)

In [6]:
train_clean = train_clean[(train_clean[['is_booking']] != 0).all(axis=1)]

In [7]:
train_clean.head()

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,hotel_country,hotel_market,hotel_cluster,srch_ci_year,srch_ci_mm,srch_ci_day,srch_ci_season,srch_ppl_cnt,stay_duration_day,search_duration_day
1,2,3,66,348,48862,2234,12,0,1,9,...,50,628,1,2014,8,29,3,2,4,17
18,2,3,66,462,41898,2454,1482,0,1,1,...,50,680,95,2014,2,22,1,3,5,49
20,2,3,66,462,41898,2454,1482,0,1,1,...,50,680,95,2014,2,22,1,2,5,49
22,2,3,66,462,41898,2454,1482,0,1,1,...,50,680,95,2014,2,24,1,2,3,51
59,2,3,66,174,40365,8456,1713,0,0,0,...,5,89,38,2014,11,11,4,2,1,12


In [8]:
print("Length of the data: {}".format(len(train_clean)))

Length of the data: 1985514


In [9]:
train_clean.to_csv('data/train_clean_booked.csv', sep=',', index=False)

### Create train data

In [10]:
read_every = 23
with open('data/train_clean.csv', 'r') as rf:
    with open('data/train_clean_1m_line.csv', 'w') as wf:
        counter = 0
        rows = 0
        for line in rf:
            if (counter == 0 or counter%read_every==0) and rows<=1000000:
                wf.write(line)
                rows+=1
            counter+=1
            
train_data = pd.read_csv('data/train_clean_1m_line.csv', delimiter = ',')
print("Length of the train data: {}".format(len(train_data)))

Length of the train data: 1000000


In [11]:
train_data = mod_df(train_data)

In [12]:
train_data.columns

Index(['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt',
       'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt',
       'hotel_continent', 'hotel_country', 'hotel_market', 'hotel_cluster',
       'srch_ci_year', 'srch_ci_mm', 'srch_ci_day', 'srch_ci_season',
       'srch_ppl_cnt', 'stay_duration_day', 'search_duration_day'],
      dtype='object')

In [13]:
train_data.to_csv('data/train_clean_1m_line_mod.csv', sep=',', index=False)

### Create validation data

In [14]:
read_every = 19
with open('data/train_clean.csv', 'r') as rf:
    with open('data/validation_1m_line.csv', 'w') as wf:
        counter = 0
        rows = 0 
        for line in rf:
            if (counter == 0 or counter%read_every==0) and rows<=1000000:
                wf.write(line)
                rows+=1
            counter+=1
            
validation_data = pd.read_csv('data/validation_1m_line.csv', delimiter = ',')
print("Length of the validation data: {}".format(len(validation_data)))

Length of the validation data: 1000000


In [15]:
validation_data = mod_df(validation_data)

In [16]:
validation_data.columns

Index(['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt',
       'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt',
       'hotel_continent', 'hotel_country', 'hotel_market', 'hotel_cluster',
       'srch_ci_year', 'srch_ci_mm', 'srch_ci_day', 'srch_ci_season',
       'srch_ppl_cnt', 'stay_duration_day', 'search_duration_day'],
      dtype='object')

In [17]:
validation_data.to_csv('data/validation_clean_1m_line_mod.csv', sep=',', index=False)

## Create test data

In [18]:
test_data = pd.read_csv('data/test.csv', delimiter = ',')

In [19]:
test_data.columns

Index(['id', 'date_time', 'site_name', 'posa_continent',
       'user_location_country', 'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id',
       'hotel_continent', 'hotel_country', 'hotel_market'],
      dtype='object')

In [20]:
test_data = mod_df(test_data)

2016-01-22 2161-10-00
2161-10-00 2015-09-27 12:17:51


In [22]:
test_data.to_csv('data/test_modified.csv', sep=',', index=False)